    Здесь буду собирать бейзлайны и немношк инфы по основным алгоритмам ML. Ехала

# <center>Read, check null, transform

In [3]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
%matplotlib inline

#загрузка данных
train_df = pd.read_csv('./data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('./data/credit_scoring_test.csv', index_col='client_id')

#выделение целевой переменной и дроп из фрейма
y = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

In [5]:
train_df.head()
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75000 entries, 0 to 74999
Data columns (total 9 columns):
DIR                      75000 non-null float64
Age                      75000 non-null float64
NumLoans                 75000 non-null int64
NumRealEstateLoans       75000 non-null int64
NumDependents            73084 non-null float64
Num30-59Delinquencies    75000 non-null int64
Num60-89Delinquencies    75000 non-null int64
Income                   60153 non-null float64
BalanceToCreditLimit     75000 non-null float64
dtypes: float64(5), int64(4)
memory usage: 5.7 MB


Замена пропусков меданными значениями

In [6]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

# <center> Decision Trees

In [7]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from ipywidgets import Image
from io import StringIO
import pydotplus #pip install pydotplus

In [16]:
#загрузка данных
train_df = pd.read_csv('./data/credit_scoring_train.csv', index_col='client_id')
test_df = pd.read_csv('./data/credit_scoring_test.csv', index_col='client_id')

#выделение целевой переменной и дроп из фрейма
y_train = train_df['Delinquent90']
train_df.drop('Delinquent90', axis=1, inplace=True)

test_df.head()
# y_test2 = test_df['Delinquent90']
# test_df2.drop('Delinquent90', axis=1, inplace=True)

,DIR,Age,NumLoans,NumRealEstateLoans,NumDependents,Num30-59Delinquencies,Num60-89Delinquencies,Income,BalanceToCreditLimit
client_id,,,,,,,,,
75000,0.488558,39.2,7,2,2.0,0,0,2866.926559,0.369443
75001,0.132810,42.3,8,0,1.0,4,0,4303.412944,1.028329
75002,1784.812905,51.5,5,1,0.0,0,0,NaN,0.081461
75003,0.538571,57.1,30,2,0.0,0,0,7672.294930,0.485850
75004,0.098539,70.1,3,0,0.0,0,0,4507.010360,0.004258


In [ ]:
train_df

In [9]:
train_df['NumDependents'].fillna(train_df['NumDependents'].median(), inplace=True)
train_df['Income'].fillna(train_df['Income'].median(), inplace=True)
test_df['NumDependents'].fillna(test_df['NumDependents'].median(), inplace=True)
test_df['Income'].fillna(test_df['Income'].median(), inplace=True)

Without Parameter tuning

In [10]:
tree = DecisionTreeClassifier(max_depth=3, random_state=17)
tree.fit(train_df, y_train)

NameError: name 'X_train' is not defined